In [71]:
from pathlib import Path
import csv
import pandas as pd

In [72]:
# loading the data

notebook_dir = Path.cwd()
# the data is located in data/raw folder
data_dir = notebook_dir.parent.absolute() / 'data_raw'
csv_dir_grouped = notebook_dir.parent.absolute() / 'clean_csvs' / 'messages_grouped_by_user'
csv_dir_separate = notebook_dir.parent.absolute() / 'clean_csvs' / 'messages_separate'

#convert txt of imessage texts to string
path_raw = data_dir / 'luigi_discord.csv'
path_csv_grouped = csv_dir_grouped / 'aarush-discord-grouped.csv'
path_csv_separate = csv_dir_separate / 'aarush-discord-separate.csv'

In [73]:
old_discord_df = pd.read_csv(path_raw)
discord_df = pd.DataFrame()
old_discord_df.to_csv('output.csv')


In [74]:
#"2023-10-28T22:28:06.2120000-07:00" -> "Mar 28, 2024  1:11:13 PM" d/m/y H/M/S
#convert the date to match the format of imsg 2024-03-25 21:51:40
df = old_discord_df.copy()

df['datetime'] = pd.to_datetime(df['Date'], utc = True)

df['datetime_pacific'] = df['datetime'].dt.tz_convert('America/Los_Angeles')

# Format the datetime column to the desired format (Y-m-d H:M:S)
df['formatted_timestamp'] = df['datetime_pacific'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Display the DataFrame
discord_df['date'] = df['formatted_timestamp']

#verify no data was lost
print(len(old_discord_df.axes[0]))
print(len(discord_df['date'].axes[0]))



160575
160575


In [75]:
old_discord_df['formatted_date'] = discord_df['date']

In [76]:
discord_df.to_csv('output.csv')
old_discord_df.to_csv('output2.csv')

In [77]:
df = old_discord_df.copy()
# Convert the 'Date' column to datetime with timezone info
df['datetime'] = pd.to_datetime(df['Date'], utc=True)

# Convert to desired format if needed
df['formatted_timestamp'] = df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Filter rows for September 15, 2022
# Convert the date string to datetime for comparison
target_date = pd.to_datetime('2022-09-15').date()
filtered_df = df[df['datetime'].dt.date == target_date]

# Print the number of rows on that date
print(f"Number of rows on September 15, 2022: {filtered_df.shape[0]}")

df.to_csv('output.csv')

Number of rows on September 15, 2022: 566


In [78]:
#convert from discord username to real names
discord_df['user'] = old_discord_df['Author'].apply(lambda x: 'Samyukta' if x == 'kurankuu' else 'Aarush')
print(discord_df.head())

                  date      user
0  2019-11-20 19:00:12  Samyukta
1  2019-11-20 19:00:15  Samyukta
2  2019-11-20 19:00:23  Samyukta
3  2019-11-20 19:00:30    Aarush
4  2019-11-20 19:01:20  Samyukta


In [79]:
#add the texts as a column
discord_df['text'] = old_discord_df['Content']


In [80]:
#add the word count as a column
# Define a function to count the number of words
def count_words(text):
    if pd.isna(text):
        return 0
    return len(text.split())

# Apply the function to the 'text' column
discord_df['word count'] = discord_df['text'].apply(count_words)

In [81]:
print(discord_df.head())

                  date      user                        text  word count
0  2019-11-20 19:00:12  Samyukta                       hello           1
1  2019-11-20 19:00:15  Samyukta                      apaphy           1
2  2019-11-20 19:00:23  Samyukta        little bunny foo foo           4
3  2019-11-20 19:00:30    Aarush  omg wow you finally accept           5
4  2019-11-20 19:01:20  Samyukta                  haha ;))))           2


In [82]:
#'date', 'user', 'text', 'word count', 'number of separated texts', 'reactions', 'game pigeon', 'platform'

discord_df['reactions'] = old_discord_df['Reactions']
discord_df['platform'] = 'Discord'
print(discord_df)

discord_df.to_csv(path_csv_separate)

                       date      user                                  text  \
0       2019-11-20 19:00:12  Samyukta                                 hello   
1       2019-11-20 19:00:15  Samyukta                                apaphy   
2       2019-11-20 19:00:23  Samyukta                  little bunny foo foo   
3       2019-11-20 19:00:30    Aarush            omg wow you finally accept   
4       2019-11-20 19:01:20  Samyukta                            haha ;))))   
...                     ...       ...                                   ...   
160570  2024-07-30 19:33:42    Aarush                         WHAT THE FUCK   
160571  2024-07-30 19:33:47    Aarush             i giggled so much at this   
160572  2024-07-30 20:40:29  Samyukta         WHY DID U GIGGLE IM TERRIFIED   
160573  2024-07-30 20:40:56    Aarush          BC OF WHAT I SAID SR THE TOP   
160574  2024-07-30 20:41:07    Aarush  WE CAN FINALLY SEND PICS OF CHILDREN   

        word count reactions platform  
0          

In [83]:
#'date', 'user', 'text', 'word count', 'number of separated texts', 'reactions', 'game pigeon', 'platform'
#group all the texts from one user that are next to each other in the csv. 
#the text column should have all the texts from that user combined into one string and separated by \n
#the number of separated texts column should have the number of rows that were merged

# Function to combine consecutive messages from the same user
def combine_consecutive_messages(df):
    # Create a column to mark groups of consecutive rows with the same user
    df['group'] = (df['user'] != df['user'].shift()).cumsum()
    
    # Group by 'user' and 'group', and aggregate the texts and count of messages
    result_df = df.groupby(['user', 'group']).agg({
        'text': lambda x: '\n'.join(x),  # Combine texts with newline separator
        'user': 'size'  # Count number of rows merged
    }).rename(columns={'user': 'number of separated texts'})
    
    # Reset index to convert 'user' and 'group' to columns
    result_df = result_df.reset_index()
    
    # Drop the 'group' column as it is no longer needed
    result_df = result_df.drop(columns=['group'])
    
    return result_df
discord_df['text'] = discord_df['text'].fillna('')
discord_df['group'] = (discord_df['user'] != discord_df['user'].shift()).cumsum()
discord_df['number of separated texts'] = discord_df['group']
result_df = discord_df.groupby(['user', 'group']).agg({
    'text': lambda x: '\n'.join(x),  # Combine texts with newline separator
    'date' : 'first',
    'word count' : 'sum',
    'user': 'size'  # Count number of rows merged
}).rename(columns={'user': 'number of separated texts'})

# Reset index to convert group columns back to regular columns
result_df = result_df.reset_index()
result_df = result_df.sort_values(by='date')
result_df = result_df.drop('group', axis = 1)
# Display the result




In [84]:
result_df['reactions'] = old_discord_df['Reactions']
result_df['platform'] = 'Discord'

In [85]:
result_df.to_csv(path_csv_grouped)